In [3]:
! pip install opencv-python

   ---------------------------------------- 0.0/38.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/38.6 MB 653.6 kB/s eta 0:01:00
   ---------------------------------------- 0.1/38.6 MB 1.2 MB/s eta 0:00:33
   ---------------------------------------- 0.2/38.6 MB 1.3 MB/s eta 0:00:30
   ---------------------------------------- 0.2/38.6 MB 1.2 MB/s eta 0:00:33
   ---------------------------------------- 0.3/38.6 MB 1.3 MB/s eta 0:00:30
   ---------------------------------------- 0.3/38.6 MB 1.3 MB/s eta 0:00:30
   ---------------------------------------- 0.4/38.6 MB 1.3 MB/s eta 0:00:30
   ---------------------------------------- 0.5/38.6 MB 1.3 MB/s eta 0:00:30
    --------------------------------------- 0.6/38.6 MB 1.4 MB/s eta 0:00:28
    --------------------------------------- 0.6/38.6 MB 1.4 MB/s eta 0:00:28
    --------------------------------------- 0.7/38.6 MB 1.4 MB/s eta 0:00:27
    --------------------------------------- 0.8/38.6 MB 1.5 MB/s eta 0:00:26
    


[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [10]:
import cv2
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.preprocessing.image import ImageDataGenerator


In [11]:
def load_and_preprocess_image(image_path, size=(128, 128)):
    image = cv2.imread(image_path)
    image = cv2.resize(image, size)
    image = image / 255.0
    return image

In [12]:
train_data = []
train_labels = []

for dirname, _, filenames in sorted(os.walk(r'C:\Users\vishnuvarthan\Downloads\archive (8)\Dataset\train')):
    for filename in filenames:
        if filename.endswith(".jpg") or filename.endswith(".jpeg"):
            image_path = os.path.join(dirname, filename)
            image = load_and_preprocess_image(image_path)
            label = 1 if dirname.split("\\")[-1] == "non-biodegradable" else 0
            train_data.append(image)
            train_labels.append(label)

train_data = np.array(train_data)
train_labels = np.array(train_labels)

test_data = []
test_labels = []

for dirname, _, filenames in sorted(os.walk(r'C:\Users\vishnuvarthan\Downloads\archive (8)\Dataset\val')):
    for filename in filenames:
        if filename.endswith(".jpg") or filename.endswith(".jpeg"):
            image_path = os.path.join(dirname, filename)
            image = load_and_preprocess_image(image_path)
            label = 1 if dirname.split("\\")[-1] == "non-biodegradable" else 0
            test_data.append(image)
            test_labels.append(label)

test_data = np.array(test_data)
test_labels = np.array(test_labels)

In [13]:
train_datagen = ImageDataGenerator(shear_range=0.2, zoom_range=0.2, horizontal_flip=True)
test_datagen = ImageDataGenerator(rescale=1./255)

In [14]:
train_dataset = tf.data.Dataset.from_tensor_slices((train_data, train_labels))
train_dataset = train_dataset.shuffle(buffer_size=1000).batch(32).prefetch(tf.data.AUTOTUNE)

test_dataset = tf.data.Dataset.from_tensor_slices((test_data, test_labels))
test_dataset = test_dataset.batch(32).prefetch(tf.data.AUTOTUNE)

In [15]:
model = Sequential()

In [16]:
model.add(Conv2D(32, (3, 3), input_shape=(128, 128, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(units=128, activation='relu'))
model.add(Dense(units=1, activation='sigmoid'))

c:\Users\vishnuvarthan\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


In [17]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [18]:
model.fit(train_dataset, epochs=25, validation_data=test_dataset)

Epoch 1/25
166/166 ━━━━━━━━━━━━━━━━━━━━ 37s 180ms/step - accuracy: 0.9691 - loss: 0.0256 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 2/25
166/166 ━━━━━━━━━━━━━━━━━━━━ 28s 169ms/step - accuracy: 1.0000 - loss: 5.3002e-21 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 3/25
166/166 ━━━━━━━━━━━━━━━━━━━━ 28s 168ms/step - accuracy: 1.0000 - loss: 2.3384e-21 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 4/25
166/166 ━━━━━━━━━━━━━━━━━━━━ 28s 170ms/step - accuracy: 1.0000 - loss: 7.2815e-21 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 5/25
166/166 ━━━━━━━━━━━━━━━━━━━━ 29s 173ms/step - accuracy: 1.0000 - loss: 3.6494e-21 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 6/25
166/166 ━━━━━━━━━━━━━━━━━━━━ 29s 172ms/step - accuracy: 1.0000 - loss: 7.0753e-21 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 7/25
166/166 ━━━━━━━━━━━━━━━━━━━━ 29s 174ms/step - accuracy: 1.0000 - loss: 2.1094e-21 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 8/25
166/166 ━━━━━━━━━━━━

In [19]:
model.save('image_classifier_model.h5')